In [1]:
!apt install aptitude swig > /dev/null
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null
!pip install mecab-python3==0.996.3 > /dev/null
!pip install pykakasi > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null



Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11.6M  100 11.6M    0     0  2120k      0  0:00:05  0:00:05 --:--:-- 2526k
./../../libexec/iconv_euc_to_utf8.sh ./Noun.verbal.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Conjunction.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Prefix.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.org.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Suffix.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Noun.adjv.csv 
./../../libexec/iconv_euc_to_utf8.sh ./Postp-col.csv 
./../../libexec/iconv_euc_to_utf

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7" -O /content/dataset.zip && rm -rf /tmp/cookies.txt
!unzip -q /content/dataset.zip -d /content

--2021-03-15 05:46:41--  https://docs.google.com/uc?export=download&confirm=nZbB&id=1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7
Resolving docs.google.com (docs.google.com)... 108.177.111.113, 108.177.111.138, 108.177.111.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.111.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-as-docs.googleusercontent.com/docs/securesc/o6nikvdipstt3v9h6fbgqbdq1203srab/useeu2priqkhfbesq4mq0222duedltf5/1615787175000/03816308716299652124/12860666482099344035Z/1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7?e=download [following]
--2021-03-15 05:46:41--  https://doc-10-as-docs.googleusercontent.com/docs/securesc/o6nikvdipstt3v9h6fbgqbdq1203srab/useeu2priqkhfbesq4mq0222duedltf5/1615787175000/03816308716299652124/12860666482099344035Z/1-4YOGJotm9OGeDUUDg7kkmke-jubBYH7?e=download
Resolving doc-10-as-docs.googleusercontent.com (doc-10-as-docs.googleusercontent.com)... 209.85.200.132, 2607:f8b0:4001:c16::84
Connec

In [3]:
import unicodedata
import string
import MeCab
import pykakasi.kakasi as kakasi

def format_text(text):
    text = unicodedata.normalize("NFKC", text)  # 全角記号をざっくり半角へ置換（でも不完全）

    # 記号を消し去るための魔法のテーブル作成
    table = str.maketrans("", "", string.punctuation  + "「」・")
    text = text.translate(table)
    text = text.translate(str.maketrans({'、' : ',', '。' : '.'}))
    return text

def getPronunciation(text):
    m_result = m.parse(text).splitlines() #mecabの解析結果の取得
    m_result = m_result[:-1] #最後の1行は不要な行なので除く

    pro = '' #発音文字列全体を格納する変数
    for v in m_result:
        if '\t' not in v: continue
        surface = v.split('\t')[0] #表層形
        p = v.split('\t')[1].split(',')[-1] #発音を取得したいとき
        #p = v.split('\t')[1].split(',')[-2] #ルビを取得したいとき
        #発音が取得できていないときsurfaceで代用
        if p == '*':
          p = surface
        pro += p

    pro = conv.do(pro) #ひらがなをカタカナに変換
    pro = format_text(pro) #余計な記号を削除

    return pro

In [4]:
import subprocess

cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))
kakasi = kakasi()
kakasi.setMode("K","a")
conv = kakasi.getConverter()

In [22]:
import os
import json
dirs = ['basic5000','precedent130','repeat500','loanword128','travel1000','countersuffix26','onomatopee300','utparaphrase512']
baseUrl = '/content/jsut_ver1.1'

In [32]:
for dir in dirs:
  out_path = os.path.join(baseUrl, dir, "correct_stt.txt")
  with open(out_path, mode='w', encoding='utf-8') as fout:

    file_path = os.path.join(baseUrl,dir,'transcript_utf8.txt')
    f = open(file_path, 'r', encoding='UTF-8')
    datalist = f.readlines()
    for data in datalist:
      data = str.split(data,":")
      wav_file = data[0] + '.wav'
      true_text = format_text(getPronunciation(data[1]))

      json_file = data[0] + '.json'
      json_file_path = os.path.join(baseUrl,dir,'json',json_file)
      with open(json_file_path, 'r', encoding='UTF-8') as fj:
        content = fj.read()
        content = json.loads(content)
        transcript = content['results'][0]['alternatives'][0]['transcript']
        estimate_text = format_text(getPronunciation(transcript))

        if true_text == estimate_text:
          fout.write(data[0] + '\n')

    f.close()